# 🚀 PostX GPU Worker

GPU Worker สำหรับสร้างภาพและวีดีโอด้วย AI

## การใช้งาน
1. เปิด GPU Runtime: Runtime > Change runtime type > GPU
2. รันทุก Cell ตามลำดับ
3. คัดลอก ngrok URL ไปใส่ในโปรแกรม PostXAgent

---

## 1️⃣ ตรวจสอบ GPU

In [ ]:
# ตรวจสอบ GPU
!nvidia-smi

import torch
print(f"\n✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("❌ ไม่พบ GPU! กรุณาเปิด GPU Runtime")

## 2️⃣ ติดตั้ง Dependencies

In [ ]:
# ติดตั้ง packages ที่จำเป็น
!pip install -q diffusers transformers accelerate safetensors
!pip install -q fastapi uvicorn websockets pyngrok
!pip install -q huggingface-hub pillow pynvml psutil

print("✅ ติดตั้งเสร็จสิ้น!")

## 3️⃣ ตั้งค่า ngrok Token

สมัครบัญชี ngrok ฟรีที่: https://ngrok.com/

แล้วคัดลอก Auth Token มาใส่ด้านล่าง

In [ ]:
# ใส่ ngrok token ของคุณที่นี่
NGROK_TOKEN = "your_ngrok_token_here"  # @param {type:"string"}

# ตั้งชื่อ Worker
WORKER_NAME = "Colab-GPU-Worker"  # @param {type:"string"}

# Master Server URL (ถ้าต้องการเชื่อมต่อกับ PostXAgent)
MASTER_URL = ""  # @param {type:"string"}

# ตั้งค่า ngrok
if NGROK_TOKEN != "your_ngrok_token_here":
    !ngrok authtoken {NGROK_TOKEN}
    print("✅ ตั้งค่า ngrok token เรียบร้อย")
else:
    print("⚠️ กรุณาใส่ ngrok token ก่อน")

## 4️⃣ สร้าง GPU Worker Code

In [ ]:
%%writefile gpu_worker.py
"""
PostX GPU Worker for Google Colab
=================================
"""
import asyncio
import logging
import torch
import io
import base64
import time
import uuid
from typing import Optional, List, Dict, Any
from dataclasses import dataclass
from PIL import Image

from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel

# Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ============================================================================
# Models
# ============================================================================

@dataclass
class GPUInfo:
    id: int
    name: str
    memory_total: float
    memory_used: float
    memory_free: float
    utilization: float = 0
    temperature: float = 0


class ImageRequest(BaseModel):
    prompt: str
    negative_prompt: str = ""
    width: int = 1024
    height: int = 1024
    steps: int = 30
    guidance_scale: float = 7.5
    seed: int = -1
    batch_size: int = 1
    model_id: str = "stabilityai/stable-diffusion-xl-base-1.0"


class TaskResponse(BaseModel):
    task_id: str
    status: str
    message: str


# ============================================================================
# GPU Monitor
# ============================================================================

def get_gpu_info() -> List[GPUInfo]:
    """Get GPU information"""
    gpus = []
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            props = torch.cuda.get_device_properties(i)
            total = props.total_memory / 1024**2
            used = torch.cuda.memory_allocated(i) / 1024**2
            free = total - used
            gpus.append(GPUInfo(
                id=i,
                name=props.name,
                memory_total=total,
                memory_used=used,
                memory_free=free,
            ))
    return gpus


# ============================================================================
# Image Generator
# ============================================================================

class ImageGenerator:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.dtype = torch.float16 if self.device == "cuda" else torch.float32
        self.pipeline = None
        self.current_model = None

    def load_model(self, model_id: str):
        if self.current_model == model_id:
            return True

        self.unload_model()
        logger.info(f"Loading model: {model_id}")

        try:
            from diffusers import StableDiffusionXLPipeline

            self.pipeline = StableDiffusionXLPipeline.from_pretrained(
                model_id,
                torch_dtype=self.dtype,
                use_safetensors=True,
                variant="fp16" if self.dtype == torch.float16 else None,
            )
            self.pipeline = self.pipeline.to(self.device)
            self.pipeline.enable_attention_slicing()
            self.current_model = model_id
            logger.info(f"Model loaded: {model_id}")
            return True
        except Exception as e:
            logger.error(f"Failed to load model: {e}")
            return False

    def unload_model(self):
        if self.pipeline:
            del self.pipeline
            self.pipeline = None
            self.current_model = None
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    def generate(self, request: ImageRequest) -> Dict[str, Any]:
        if not self.load_model(request.model_id):
            raise Exception("Failed to load model")

        seed = request.seed if request.seed >= 0 else torch.randint(0, 2**32-1, (1,)).item()
        generator = torch.Generator(device=self.device).manual_seed(seed)

        start_time = time.time()

        output = self.pipeline(
            prompt=request.prompt,
            negative_prompt=request.negative_prompt or None,
            width=request.width,
            height=request.height,
            num_inference_steps=request.steps,
            guidance_scale=request.guidance_scale,
            num_images_per_prompt=request.batch_size,
            generator=generator,
        )

        generation_time = time.time() - start_time

        # Convert to base64
        images_b64 = []
        for img in output.images:
            buffer = io.BytesIO()
            img.save(buffer, format="PNG")
            buffer.seek(0)
            images_b64.append(base64.b64encode(buffer.read()).decode())

        return {
            "images": images_b64,
            "seed": seed,
            "generation_time": generation_time,
            "model_id": request.model_id,
        }


# Global generator
generator = ImageGenerator()

# ============================================================================
# FastAPI App
# ============================================================================

app = FastAPI(title="PostX GPU Worker", version="1.0.0")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# State
task_results = {}


@app.get("/")
async def root():
    return {"service": "PostX GPU Worker", "status": "running"}


@app.get("/health")
async def health():
    return {"status": "healthy"}


@app.get("/status")
async def get_status():
    gpus = get_gpu_info()
    return {
        "status": "online",
        "gpu_count": len(gpus),
        "gpus": [{
            "id": g.id,
            "name": g.name,
            "memory_total_gb": g.memory_total / 1024,
            "memory_free_gb": g.memory_free / 1024,
        } for g in gpus],
        "total_vram_gb": sum(g.memory_total for g in gpus) / 1024,
        "free_vram_gb": sum(g.memory_free for g in gpus) / 1024,
        "current_model": generator.current_model,
    }


@app.post("/generate/image")
async def generate_image(request: ImageRequest):
    task_id = str(uuid.uuid4())
    
    try:
        result = await asyncio.to_thread(generator.generate, request)
        task_results[task_id] = {
            "status": "completed",
            "result": result
        }
        return {
            "task_id": task_id,
            "status": "completed",
            "result": result
        }
    except Exception as e:
        task_results[task_id] = {
            "status": "failed",
            "error": str(e)
        }
        raise HTTPException(status_code=500, detail=str(e))


@app.get("/task/{task_id}")
async def get_task(task_id: str):
    if task_id not in task_results:
        raise HTTPException(status_code=404, detail="Task not found")
    return task_results[task_id]


@app.post("/model/load")
async def load_model(model_id: str):
    success = await asyncio.to_thread(generator.load_model, model_id)
    if success:
        return {"message": f"Model {model_id} loaded"}
    raise HTTPException(status_code=500, detail="Failed to load model")


@app.post("/model/unload")
async def unload_model():
    generator.unload_model()
    return {"message": "Model unloaded"}


if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8080)

## 5️⃣ เริ่ม GPU Worker

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from pyngrok import ngrok
import threading
import uvicorn

# Import worker
from gpu_worker import app

# Start uvicorn in background
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8080, log_level="info")

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

# Wait for server to start
import time
time.sleep(3)

# Start ngrok tunnel
public_url = ngrok.connect(8080)

print("\n" + "="*60)
print("🚀 PostX GPU Worker พร้อมใช้งาน!")
print("="*60)
print(f"\n📡 Public URL: {public_url}")
print(f"\n📋 คัดลอก URL นี้ไปใส่ในโปรแกรม PostXAgent")
print("\n" + "="*60)

# Test endpoint
import requests
try:
    response = requests.get(f"{public_url}/status")
    status = response.json()
    print(f"\n✅ GPU: {status['gpus'][0]['name'] if status['gpus'] else 'N/A'}")
    print(f"✅ VRAM: {status['total_vram_gb']:.1f} GB")
except:
    print("⚠️ ไม่สามารถเชื่อมต่อได้ รอสักครู่แล้วลองใหม่")

## 6️⃣ ทดสอบสร้างภาพ

In [ ]:
import requests
import base64
from PIL import Image
import io
from IPython.display import display

# ทดสอบสร้างภาพ
prompt = "A beautiful sunset over the ocean, photorealistic, 8k"  # @param {type:"string"}

print("🎨 กำลังสร้างภาพ... (อาจใช้เวลา 30-60 วินาที)")

response = requests.post(
    f"{public_url}/generate/image",
    json={
        "prompt": prompt,
        "width": 1024,
        "height": 1024,
        "steps": 30,
    }
)

if response.status_code == 200:
    result = response.json()
    print(f"\n✅ สร้างภาพเสร็จใน {result['result']['generation_time']:.1f} วินาที")
    print(f"Seed: {result['result']['seed']}")
    
    # แสดงภาพ
    for img_b64 in result['result']['images']:
        img_data = base64.b64decode(img_b64)
        img = Image.open(io.BytesIO(img_data))
        display(img)
else:
    print(f"❌ เกิดข้อผิดพลาด: {response.text}")

## 📌 หมายเหตุ

- Colab จะตัดการเชื่อมต่อหลังจากไม่มีการใช้งาน ~30 นาที
- Colab Pro จะให้ใช้งานได้นานกว่า และ GPU แรงกว่า
- หากต้องการใช้งานต่อเนื่อง แนะนำให้เช่า GPU จาก Runpod, Vast.ai หรือ Lambda Labs

---

Made with ❤️ by PostXAgent Team